# Computing Energy Conditions

This notebook demonstrates how to compute and analyze energy conditions for warp drive spacetimes. Energy conditions are fundamental constraints that determine whether a spacetime solution requires exotic matter (matter with negative energy density) to exist.

The four main energy conditions are:
- **Null Energy Condition (NEC)**: The weakest condition, requiring that energy density is non-negative along null geodesics
- **Weak Energy Condition (WEC)**: Energy density is non-negative for all timelike observers
- **Dominant Energy Condition (DEC)**: Energy cannot flow faster than light
- **Strong Energy Condition (SEC)**: Gravity is always attractive

Warp drive spacetimes typically violate one or more of these conditions, indicating the need for exotic matter.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Import WarpFactory modules
from warpfactory.metrics.alcubierre import get_alcubierre_metric
from warpfactory.solver.energy import get_energy_tensor
from warpfactory.analyzer.energy_conditions import get_energy_conditions

## Get Metric and Energy Tensor

First, we create an Alcubierre warp drive metric and compute its stress-energy tensor using Einstein's field equations.

In [ ]:
# Define Alcubierre metric parameters
grid_size = [1, 20, 20, 20]  # [time, x, y, z]
world_center = [(gs + 1) / 2 for gs in grid_size]  # Center of the grid
velocity = 0.5  # Velocity in units of c
R = 5.0  # Radius of the warp bubble
sigma = 0.5  # Width of the transition region

print(f"Creating Alcubierre metric with:")
print(f"  Grid size: {grid_size}")
print(f"  Velocity: {velocity}c")
print(f"  Bubble radius: {R}")
print(f"  Sigma: {sigma}")

# Create the Alcubierre metric
metric = get_alcubierre_metric(
    grid_size=grid_size,
    world_center=world_center,
    velocity=velocity,
    R=R,
    sigma=sigma
)

print(f"\nMetric created: {metric.name}")
print(f"  Shape: {metric.shape}")
print(f"  Index type: {metric.index}")

In [ ]:
# Compute the stress-energy tensor from the metric
print("Computing stress-energy tensor...")
energy_tensor = get_energy_tensor(metric)

print(f"\nEnergy tensor computed: {energy_tensor.tensor_type}")
print(f"  Index type: {energy_tensor.index}")
print(f"  Shape: {energy_tensor.shape}")

## Get Energy Conditions Map

Now we evaluate all four energy conditions across the spacetime. For each point, the function samples multiple null or timelike vectors and returns the most violating evaluation.

A negative value indicates violation of the energy condition at that point.

In [ ]:
# Evaluate all four energy conditions
print("Evaluating energy conditions...\n")

print("Computing Null Energy Condition...")
null_energy_condition, _, _ = get_energy_conditions(
    energy_tensor, metric, "Null"
)

print("Computing Weak Energy Condition...")
weak_energy_condition, _, _ = get_energy_conditions(
    energy_tensor, metric, "Weak"
)

print("Computing Strong Energy Condition...")
strong_energy_condition, _, _ = get_energy_conditions(
    energy_tensor, metric, "Strong"
)

print("Computing Dominant Energy Condition...")
dominant_energy_condition, _, _ = get_energy_conditions(
    energy_tensor, metric, "Dominant"
)

print("\nAll energy conditions computed!")

## Plot Energy Conditions

Visualize the energy condition violations in a 2D slice through the warp bubble. We'll take a slice at z=10 (the middle of the grid).

In [ ]:
# Extract 2D slices at z=10 (middle of the grid) and time=0
z_slice = 10
time_slice = 0

# Create figure with 2x2 subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 12))
fig.suptitle('Energy Conditions for Alcubierre Warp Drive', fontsize=16, y=0.995)

# Plot Null Energy Condition
ax = axes[0, 0]
im1 = ax.imshow(
    null_energy_condition[time_slice, :, :, z_slice].T,
    origin='lower',
    cmap='RdBu_r',
    aspect='auto'
)
ax.set_title('Null Energy Condition (NEC)', fontsize=12, fontweight='bold')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(im1, ax=ax, label='NEC Value')

# Plot Weak Energy Condition
ax = axes[0, 1]
im2 = ax.imshow(
    weak_energy_condition[time_slice, :, :, z_slice].T,
    origin='lower',
    cmap='RdBu_r',
    aspect='auto'
)
ax.set_title('Weak Energy Condition (WEC)', fontsize=12, fontweight='bold')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(im2, ax=ax, label='WEC Value')

# Plot Strong Energy Condition
ax = axes[1, 0]
im3 = ax.imshow(
    strong_energy_condition[time_slice, :, :, z_slice].T,
    origin='lower',
    cmap='RdBu_r',
    aspect='auto'
)
ax.set_title('Strong Energy Condition (SEC)', fontsize=12, fontweight='bold')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(im3, ax=ax, label='SEC Value')

# Plot Dominant Energy Condition
ax = axes[1, 1]
im4 = ax.imshow(
    dominant_energy_condition[time_slice, :, :, z_slice].T,
    origin='lower',
    cmap='RdBu_r',
    aspect='auto'
)
ax.set_title('Dominant Energy Condition (DEC)', fontsize=12, fontweight='bold')
ax.set_xlabel('X')
ax.set_ylabel('Y')
plt.colorbar(im4, ax=ax, label='DEC Value')

plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Blue regions (negative values) indicate violation of the energy condition")
print("- Red regions (positive values) indicate satisfaction of the energy condition")
print("- Violations indicate the need for exotic matter at those locations")

## Analyze Violation Statistics

Let's compute some statistics about the energy condition violations.

In [ ]:
# Function to compute violation statistics
def compute_violation_stats(condition_map, condition_name):
    """Compute statistics about energy condition violations"""
    # Remove NaN values
    valid_values = condition_map[~np.isnan(condition_map)]
    
    # Count violations (negative values)
    violations = valid_values < 0
    num_violations = np.sum(violations)
    total_points = len(valid_values)
    violation_percentage = (num_violations / total_points) * 100
    
    # Get min/max violation values
    min_value = np.min(valid_values)
    max_value = np.max(valid_values)
    
    print(f"\n{condition_name}:")
    print(f"  Violation points: {num_violations}/{total_points} ({violation_percentage:.2f}%)")
    print(f"  Min value (worst violation): {min_value:.6e}")
    print(f"  Max value: {max_value:.6e}")
    
    return {
        'condition': condition_name,
        'num_violations': num_violations,
        'total_points': total_points,
        'violation_percentage': violation_percentage,
        'min_value': min_value,
        'max_value': max_value
    }

# Compute statistics for all conditions
print("Energy Condition Violation Statistics:")
print("=" * 60)

null_stats = compute_violation_stats(null_energy_condition, "Null Energy Condition (NEC)")
weak_stats = compute_violation_stats(weak_energy_condition, "Weak Energy Condition (WEC)")
strong_stats = compute_violation_stats(strong_energy_condition, "Strong Energy Condition (SEC)")
dominant_stats = compute_violation_stats(dominant_energy_condition, "Dominant Energy Condition (DEC)")

print("\n" + "=" * 60)
print("\nKey Findings:")
print("- The Alcubierre warp drive requires exotic matter (negative energy density)")
print("- Violations are localized to the warp bubble region")
print("- The Strong Energy Condition is typically the most violated")

## Returning Vectors: Detailed Analysis

We can also examine the energy condition evaluations for individual vectors at specific points. This shows how different observer orientations experience the energy density.

Let's analyze the Null Energy Condition at a single point with multiple null vectors.

In [ ]:
# Compute NEC with vector field information
num_angular_vec = 100
num_time_vec = 10

print(f"Computing NEC with detailed vector information...")
print(f"  Angular vectors: {num_angular_vec}")
print(f"  Time vectors: {num_time_vec}")

null_map, null_vec_map, vector_field = get_energy_conditions(
    energy_tensor,
    metric,
    "Null",
    num_angular_vec=num_angular_vec,
    num_time_vec=num_time_vec,
    return_vec=True
)

print(f"\nVector field shape: {vector_field.shape}")
print(f"Null vector map shape: {null_vec_map.shape}")
print(f"  - Dimensions: [time, x, y, z, num_vectors]")

## Visualize Vector-Specific Violations

Plot the NEC violation for each sampled vector direction at a specific point in spacetime. This shows how the energy condition depends on the observer's orientation.

In [ ]:
# Normalize the violation amount for plotting
null_vec_map_norm = null_vec_map / np.max(np.abs(null_vec_map))

# Define the spacetime point to sample (inside the warp bubble)
sample_point = [0, 5, 10, 10]  # [t, x, y, z]

print(f"Analyzing vectors at point: t={sample_point[0]}, x={sample_point[1]}, "
      f"y={sample_point[2]}, z={sample_point[3]}")

# Create 3D quiver plot
fig = plt.figure(figsize=(12, 10))
ax = fig.add_subplot(111, projection='3d')

# Plot each null vector, colored by violation amount
for i in range(vector_field.shape[1]):
    # Get spatial components of the vector
    vx = vector_field[1, i]
    vy = vector_field[2, i]
    vz = vector_field[3, i]
    
    # Get the violation amount for this vector at the sample point
    violation = null_vec_map_norm[
        sample_point[0], sample_point[1], sample_point[2], sample_point[3], i
    ]
    
    # Color based on violation (blue=violation, red=satisfied)
    if violation < 0:
        color = 'blue'
        alpha = min(abs(violation), 1.0)
    else:
        color = 'red'
        alpha = min(violation, 1.0) * 0.3
    
    # Plot the vector
    ax.quiver(0, 0, 0, vx, vy, vz, 
              color=color, alpha=alpha, 
              length=abs(violation), arrow_length_ratio=0.3)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('Normalized Null Energy Condition Violation\nFor Different Observer Orientations',
             fontsize=14, fontweight='bold')

# Add legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], color='blue', linewidth=3, label='Violation (negative)'),
    Line2D([0], [0], color='red', linewidth=3, label='Satisfied (positive)')
]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

print("\nVisualization shows:")
print("- Blue vectors: Directions where NEC is violated")
print("- Red vectors: Directions where NEC is satisfied")
print("- Vector length: Magnitude of violation/satisfaction (normalized)")

## Summary

This notebook demonstrated:

1. **Creating a warp drive metric** - We generated an Alcubierre warp drive spacetime with specified parameters

2. **Computing the stress-energy tensor** - Using Einstein's field equations, we derived the matter distribution required for the warp drive

3. **Evaluating energy conditions** - We computed all four energy conditions (Null, Weak, Strong, Dominant) across the spacetime

4. **Visualizing violations** - We created 2D slices showing where each energy condition is violated

5. **Analyzing vector-specific behavior** - We examined how energy conditions vary with observer orientation

### Key Results:

- The Alcubierre warp drive violates multiple energy conditions
- Violations are localized to the region around the warp bubble boundary
- The extent of violation depends on the warp drive parameters (velocity, radius, sigma)
- Energy condition violations indicate the need for exotic matter with negative energy density

### Physical Interpretation:

Energy condition violations are signatures of exotic matter requirements. The Alcubierre drive requires negative energy density to function, which is one of the major theoretical and practical challenges for implementing such a propulsion system. Understanding where and how severely these conditions are violated helps inform efforts to minimize exotic matter requirements through metric engineering.